In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
import time
from selenium.webdriver.chrome.service import Service


def get_linkedin_data(first_name, last_name):
    url = f"https://www.linkedin.com/search/results/all/?keywords={first_name}%20{last_name}&origin=GLOBAL_SEARCH_HEADER&sid=(%3AB={first_name}%20{last_name}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    s = Service("C:/Users/Prince Gupta/Downloads/chromedriver-win64/chromedriver.exe")
    driver = webdriver.Chrome(service=s)

    driver.get("https://www.linkedin.com/pub/dir/+/+?trk=guest_homepage-basic_guest_nav_menu_people")
    profile_listing = driver.find_elements(By.XPATH, '//*[@id="main-content"]/section[2]/ul/li')


    if not profile_listing:  
        print(f"No profiles found for {first_name} {last_name}")
        return []

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    profiles = soup.find_all("li", class_='reusable-search__result-container')

    data = []

    for profile in profiles[:5]:
        name = profile.find("span", class_="name actor-name").get_text().strip()
        occupation = profile.find("span", class_="subline-level-1").get_text().strip()
        location = profile.find("span", class_="subline-level-2").get_text().strip()
        data.append({"Name": name, "Occupation": occupation, "Location": location})

    return data


def save_to_csv(data, filename):
    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)
    print(f"Data saved to {filename} successfully.")


if __name__ == "__main__":
    first_name = input("Enter first name: ")
    last_name = input("Enter last name: ")

    linkedin_data = get_linkedin_data(first_name, last_name)
    save_to_csv(linkedin_data, "pepole_detail.csv")
